# Обработка отсутствующих данных

Разница между данными, которые представлены во многих руководствах, и данными реального мира состоит в том, что данные реального мира достаточно редко бывают "чистыми" и однородными. В частности, множество интересных наборов данных будут иметь некоторое количество отсутствующих данных. Чтобы сделать проблему еще более сложной, представьте что различные источники данных могут обозначать отсутствующие данные разными способами.

В данной секции мы будем обсуждать некоторые общие соображения об отсутствующих данных, обсудим как Pandas выбирает как представить их и продемонстрируем некоторые встроенные в Pandas инструменты для обработки отсутствующих данных в Python. Здесь и далее, когда мы говорим об отсутствующих данных, мы, в общем, подразумеваем их как следующие значения: `null`, `NaN` или `NA`

## Компромисы в соглашениях об отсутствующих данных

В целом существуют два подхода для обозначения отсутствующих данных:

* использование _маски_ (_mask_) для глобального подхода при обозначении отсутствующих данных;
* использование _контрольного значения_ (_sentinel value_), которое обозначает отсутствующие данные.

При использовании _маски_ может применяться как абсолютно отдельные массив с булевыми значениями для обозначения отсутствующих данных, так и использование одного из битов значения для указания на отсутствующее значение.

При использовании _контрольного значения_ могут использоваться как специальные значения, например -9999 для обозначения отсутствующего целочисленного значения, так и использование специальных значений, например `NaN`.

## Отсутствующие данные в Pandas

Способ, которым обозначаются отсутствующие данные в Pandas, ограничен тем, что Pandas построен поверх NumPy и, таким образом, зависит от того как отсутствующие данные обозначаются в NumPy.

Учитывая все ограничения, Pandas выбрал использование _контрольных значений_ (sentinel) для обозначения отсутствующих данных и, в дальнейшем, полагается на два уже существующих в Python значения для отсутствующих данных: `NaN` для отсутствующих данных с плавающей точкой и `None`. Такой подход имеет, как мы увидим, некоторые побочные эффекты, но, на практике, является хорошим компромисов для большинства возможных случаев.

### `None`: способ Python для отсутствующх данных

Первое контрольное значение, используемов в Pandas - `None`, Python синглтон-объект, который часто используется для обозначения отсутствующих данных в коде на Python. Ввиду того, что это объект Python, `None` не может быть использован в абсолютно любых массивах NumPy/Pandas, но только в тех массивах, в которых типом данных является `object`, т.е. в массивах объектов Python:

In [3]:
import numpy as np
import pandas as pd

In [4]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

`dtype=object` означает что лучший тип для представления данных, который NumPy смог выбрать для содержимого массива, это объекты Python. В то время как такой тип массива является пригодным для определённых целей, любые операции с данными будут выполняться на уровне Python, с большим количеством накладных расходов чем типичные операции на массивах "родных" (для NumPy) типов:

In [5]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
40.9 ms ± 758 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
604 µs ± 47.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)



In [6]:
# На массивах, содержащих значения None, нельзя выполнять операции типа sum() или min()
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### `NaN`: отсутствующие числовые данные

`NaN` - это специальное значение с плавающей запятой, определяемое всеми системами, которые используют стандарт IEEE для представления значений с плавающей запятой.

In [8]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Заметьте, что `NumPy` выбрал для данного массива нативный тип для чисел с плавающей запятой: это значит, что в отличие от массива с `object` из предыдущего раздела, настоящий массив поддерживает быстрые операции, выполняемые на уровне скомпилированного кода. Необходимо отметить, что вне зависимости от операции, результатом арифметических операций с `NaN` всегда будет другой `NaN`:

In [9]:
1 + np.nan

nan

In [10]:
0 * np.nan

nan

In [11]:
# функции агрегирования работают, но не являются полезными
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

`NumPy` предлагает некоторые специальные функции для агрегирования, которые пропускают значения `NaN`:

In [12]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

`NaN` только для объектов с плавающей запятой, не существует аналога `NaN` для целочисленных объектов, строк или любых других типов.

### `NaN` и `None` в Pandas

`NaN` и `None` поддерживаются Pandas и практически взаимозаменяемы, поддерживается конвертирование между этими значениями там, где возможно:

In [16]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Для типов не имеющих контрольных значений, Pandas автоматически осуществляет приведение типов, когда встречает отсутствующие значения.
Например, если мы установим значение в целочисленном массиве в `np.nan`, весь массив будет автоматически приведен к типу с плавающей запятой для возможности обработки отсутствующего значения:

In [17]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [18]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Следующая таблица отображает соглашения о приведении типов в Pandas, в случаях когда встречаются отсутствующие значения:

|Typeclass     | Conversion when storing NAs | NA sentinel value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an `object` dtype.

## Операции со значениями Null

Как мы уже заметили, Pandas, для обозначения отсутствующих или null значений, использует `None` и `NaN` практически взаимозаменямо. Для облегчения использования данного соглашения, существуют несколько полезных методов для обнаружения, удаления и замены значений `null` в данных Pandas, такие как:

`isnull()`: Generate a boolean mask indicating missing values    
`notnull()`: Opposite of `isnull()`    
`dropna()`: Return a filtered version of the data    
`fillna()`: Return a copy of the data with missing values filled or imputed   

### Нахождение значений `null`

Объекты Pandas имеют два полезных метода для опеределения null значений: `isnull()` и `notnull()`. Оба возвращают булевую маску для всего набора. Например:

In [21]:
data = pd.Series([1, np.nan, 'hello', None])

In [23]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

Как указано в [Data Indexing and Selection](https://jakevdp.github.io/PythonDataScienceHandbook/03.02-data-indexing-and-selection.html), булевы маски могут быть напрямую использованы в качестве индекся для объектов `Series` и `DataFrame`:

In [24]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Удаление пустых значений

В дополнение к маскированию, использованному ранее, существуют другие удобные методы, `dropna()` - удаляет пустые значения и `fillna()` - заполняет пустыми значениями. Для объектов `Series` результат вполне понятный:

In [26]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [27]:
data.dropna()

0        1
2    hello
dtype: object

Для `DataFrame` возможностей больше, давайте рассмотрим следующий `DataFrame`:

In [30]:
df = pd.DataFrame([[1, np.nan, 2], [2, 3, 5], [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Мы не можем удалить одиночные значения из `DataFrame`; мы может только удалить весь ряд или всю колонку. В зависимости от приложения, вы можете выбрать одно или другое, так что `dropna()` предлагает набор возможных путей для `DataFrame`:

In [32]:
# удаляет все ряды с пустыми значениями
df.dropna()

,0,1,2
1,2.0,3.0,5


In [34]:
# удаляет все колонки с пустыми значениями
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Такие подходы удаляют, в том числе, и нужные данные; вы можете быть заинтересованы в удалении рядов или колонок, которые полностью состоят из пустых значений или с большинством пустых значений. Этого можно достигнуть с помощью аргументов `how` и `thresh`, которые позволяют более точно контролировать количество пустых значений, которые допустимы.

Значение по умолчанию `how='any'`, такое, что полностью удаляется любой ряд или колонка (в зависимости от оси), содержащий пустое значение. Вы также можете указать `how='all'`, чтобы удалить только те ряды/колонки, которые полностью состоят из пустых значений:

In [35]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [37]:
# удалить колонки, в которых все значения пустые
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Для более детального управления параметр `thresh` позволяет указать минимальное количество не пустых значений чтобы сохранить ряд/колонку:

In [39]:
# удалить все ряды, в которых менее 3 не пустых значений
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Заполнение пустых значений

Иногда необходимо заполнить пустые значения каким-либо другим значением, для этого используется метод `fillna()`.

In [41]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [43]:
# заменаем NaN на 0.0
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Мы можем указать прямое заполнение для распространения предыдущего значения вперед:

In [46]:
# forward-fill
# data.fillna(method='ffill') - этот метод устаревший!!!
# пустые значения заполняются новыми начиная с 1.0 и увеличивающимися на 1 для каждого последующего заполняемого значения.
data.ffill()

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [48]:
# back-fill
# data.fillna(method='bfill')
data.bfill()

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Для `DataFrame` все опции аналогичны, но мы также можем указать ось, вдоль которой осуществляется заполнение:

In [49]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [55]:
# заполнять пустые значения вдоль колонок
df.ffill(axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Обратите внимание, что если предыдущее пустое значение недоступно во время прямого заполнения, то пустое значение остается.